<a href="https://colab.research.google.com/github/bayuyap/Fuzzy_Restoran/blob/main/Fuzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Membaca file excel dan membuat output dari file 

In [ ]:
import pandas as pd

def ReadExcelFile(name):
  df = pd.read_excel(name)
  return df

x = ReadExcelFile("restoran.xlsx")
x

,id,pelayanan,makanan
0,1,58,7
1,2,54,1
2,3,98,2
3,4,52,4
4,5,11,4
...,...,...,...
95,96,30,1
96,97,25,3
97,98,27,10
98,99,8,6


# Membership Function

In [ ]:
def UpTrapesium(x, a, b):
  if x <= a:
    return 0
  elif a < x and x <= b:
    return (x-a)/(b-a)
  else:
    return 1

def DownTrapesium(x, c, d):
  if x <= c:
    return 1
  elif c < x and x <= d:
    return (d-x)/(d-c)
  else:
    return 0

def TrapesiumGrafik(x, a, b, c, d):
  if x <= a or x >= d:
    return 0
  elif a < x and x < b:
    return (x-a)/(b-a)
  elif b <= x and x <= c:
    return 1
  else:
    return -(x-d)/(d-c)

In [ ]:
def pelayananSet(pelayanan, a, b, c, d):
  crips = {'bad': 0 , 'average': 0, 'best':0}
  assert a < b, "nilai a > b"
  assert b < c, "nilai b > c"
  assert c < d, "nilai c > d"
  crips['bad'] = DownTrapesium(pelayanan,a, b)
  crips['average'] = TrapesiumGrafik(pelayanan, a, b, c, d)
  crips['best'] = UpTrapesium(pelayanan, c, d)
  return crips

In [ ]:
def makananSet(makanan, a, b, c, d):
  crips = {'bad': 0 , 'average': 0, 'best':0}
  assert a < b, "nilai a > b"
  assert b < c, "nilai b > c"
  assert c < d, "nilai c > d"
  crips['bad'] = DownTrapesium(makanan,a, b)
  crips['average'] = TrapesiumGrafik(makanan, a, b, c, d)
  crips['best'] = UpTrapesium(makanan, c, d)
  return crips

# Aturan Inferensi, Fuzzifikasi dan Defuzzifikasi

Pembuatan Aturan Inferensi

In [ ]:
from math import ceil

def checking(a, b):
  return ceil(a) and ceil(b)

def PropernessCrips(pelayanan, makanan):
  crips = {'buruk': 0, 'baik': 0}
  if checking(pelayanan['bad'], makanan['bad']):
    crips['buruk'] = max(crips['buruk'], min(pelayanan['bad'], makanan['bad']))
  if checking(pelayanan['bad'], makanan['average']):
    crips['buruk'] = max(crips['buruk'], min(pelayanan['bad'], makanan['average']))
  if checking(pelayanan['bad'], makanan['best']):
    crips['buruk'] = max(crips['buruk'], min(pelayanan['bad'], makanan['best']))
  if checking(pelayanan['average'], makanan['bad']):
    crips['buruk'] = max(crips['buruk'], min(pelayanan['average'], makanan['bad']))
  if checking(pelayanan['average'], makanan['average']):
    crips['baik'] = max(crips['baik'], min(pelayanan['average'], makanan['average']))
  if checking(pelayanan['average'], makanan['best']):
    crips['baik'] = max(crips['baik'], min(pelayanan['average'], makanan['best']))
  if checking(pelayanan['best'], makanan['bad']):
    crips['buruk'] = max(crips['buruk'], min(pelayanan['best'], makanan['bad']))
  if checking(pelayanan['best'], makanan['average']):
    crips['baik'] = max(crips['baik'], min(pelayanan['best'], makanan['average']))
  if checking(pelayanan['best'], makanan['best']):
    crips['baik'] = max(crips['baik'], min(pelayanan['best'], makanan['best']))
  return crips

In [ ]:
def Fuzzifikasi(pelayanan, makanan, paramPelayanan, paramFood):
  allCrips = {'pel':[], 'food':[]}
  
  for i in range(len(pelayanan)):
    allCrips['pel'].append(pelayananSet(pelayanan[i], paramPelayanan['a'],
                                     paramPelayanan['b'], paramPelayanan['c'], 
                                     paramPelayanan['d']))
    allCrips['food'].append(makananSet(makanan[i], paramFood['a'],
                                     paramFood['b'], paramFood['c'], 
                                     paramFood['d']))

  return allCrips

In [ ]:
def Inferensi(allCrips):
  propernessCrips = []

  for i in range(len(allCrips['pel'])):
    propernessCrips.append(PropernessCrips(allCrips['pel'][i],
                                           allCrips['food'][i]))
  
  return propernessCrips

In [ ]:
def Centroid(crips, a, b):
  score = 0
  down = 0
  for i in range(101):
    buruk = min(crips['buruk'], DownTrapesium(i,a, b))
    baik = min(crips['baik'], UpTrapesium(i,a, b))
    score +=  i * max(buruk, baik)
    down +=  max(buruk, baik)
  return score / down

In [ ]:
def Defuzzifikasi(allCrips, id):
  ans = []
  
  for i in range(len(allCrips)):
    crips = allCrips[i]
    sc = Centroid(crips, 40, 80)
    ans.append({'id':id[i], 'score':sc})

  ans = sorted(ans, key= lambda x : x['score'], reverse=True)
  return ans

# Main Program

In [ ]:
# preprocessing Data
df = ReadExcelFile("restoran.xlsx")
pelayanan = df['pelayanan'].values
makanan = df['makanan'].values
id = df['id'].values

# Setup parameter grafik pelayanan dan makanan
paramservice = {'a':40, 'b':50, 'c':70, 'd':80}
paramfood = {'a':4, 'b':5, 'c':7, 'd':8}

# Fuzzifikasi 
allCrips = Fuzzifikasi(pelayanan, makanan, paramservice, paramfood)

# Interferensi
allCripsInferensi = Inferensi(allCrips)

# Defuzzifikasi
ans = Defuzzifikasi(allCripsInferensi, id)
for a in ans[:10]:
  print(a)

# Write to excel
df1 = pd.DataFrame( [i['id'] for i in ans[:10]],
                   columns=['Id'])
df1.to_excel("peringkat.xls", index = False, header=False)

{'id': 1, 'score': 78.60493827160494}
{'id': 6, 'score': 78.60493827160494}
{'id': 7, 'score': 78.60493827160494}
{'id': 16, 'score': 78.60493827160494}
{'id': 24, 'score': 78.60493827160494}
{'id': 25, 'score': 78.60493827160494}
{'id': 27, 'score': 78.60493827160494}
{'id': 39, 'score': 78.60493827160494}
{'id': 42, 'score': 78.60493827160494}
{'id': 54, 'score': 78.60493827160494}
